# Nesesary Imports

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["NVIDIA_API_KEY"]=os.getenv('NVIDIA_API_KEY')
os.environ['LANGSMITH_API_KEY']=os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING']=os.getenv('LANGSMITH_TRACING')
os.environ['LANGSMITH_PROJECT']=os.getenv('LANGSMITH_PROJECT',"Your-proejct-name")

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../langchain_crawl_results.csv')
df.head()

,Type,URL
0,page,https://python.langchain.com/api_reference/mon...
1,page,https://python.langchain.com/v0.2/api_referenc...
2,page,https://python.langchain.com/docs/contributing...
3,page,https://api.python.langchain.com/en/latest/cha...
4,page,https://python.langchain.com/api_reference/mon...


In [8]:
list_of_urls = (df[df['Type']=='page'].URL).tolist()

In [15]:
from langchain.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(list_of_urls[:10])

In [16]:
docs = loader.load()

In [18]:
from langchain.document_loaders import WebBaseLoader

web_loader = WebBaseLoader(list_of_urls[:10])

web_docs = web_loader.load()

In [22]:
docs[5].page_content

'This is a legacy site. Please use the latest v0.2 and v0.3 API references instead.\n\nSource code for langchain_core.utils.json\n\nfrom __future__ import annotations\n\nimport json\nimport re\nfrom typing import Any, Callable, List\n\nfrom langchain_core.exceptions import OutputParserException\n\n\ndef _replace_new_line(match: re.Match[str]) -> str:\n    value = match.group(2)\n    value = re.sub(r"\\n", r"\\\\n", value)\n    value = re.sub(r"\\r", r"\\\\r", value)\n    value = re.sub(r"\\t", r"\\\\t", value)\n    value = re.sub(r\'(?<!\\\\)"\', r"\\"", value)\n\n    return match.group(1) + value + match.group(3)\n\n\ndef _custom_parser(multiline_string: str) -> str:\n    """\n    The LLM response for `action_input` may be a multiline\n    string containing unescaped newlines, tabs or quotes. This function\n    replaces those characters with their escaped counterparts.\n    (newlines in JSON must be double-escaped: `\\\\n`)\n    """\n    if isinstance(multiline_string, (bytes, bytearr

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
documents = spliter.split_documents(docs)
documents

[Document(metadata={'source': 'https://python.langchain.com/api_reference/mongodb/retrievers/pipelines/pipelines/retrievers/langchain_mongodb.retrievers.hybrid_search.MongoDBAtlasHybridSearchRetriever.html'}, page_content="langchain-mongodb: 0.6.0#\n\nIntegrate your operational database and vector search in a single, unified, fully managed platform with full vector database capabilities on MongoDB Atlas.\n\nStore your operational data, metadata, and vector embeddings in oue VectorStore, MongoDBAtlasVectorSearch. Insert into a Chain via a Vector, FullText, or Hybrid Retriever.\n\nagent_toolkit#\n\nClasses\n\nagent_toolkit.database.MongoDBDatabase (...) Wrapper around a MongoDB database. agent_toolkit.tool.BaseMongoDBDatabaseTool Base tool for interacting with a MongoDB database. agent_toolkit.tool.InfoMongoDBDatabaseTool Tool for getting metadata about a MongoDB database. agent_toolkit.tool.ListMongoDBDatabaseTool Tool for getting collection names. agent_toolkit.tool.QueryMongoDBChecker

In [27]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
# nvidia/nv-embedcode-7b-v1 ---> Mistralbased code optimized
# nvidia/llama-3.2-nv-embedqa-1b-v2 ---> Llama based QA optimized
embeddings_model = NVIDIAEmbeddings(
    model="nvidia/llama-3.2-nv-embedqa-1b-v2", 
    api_key=os.getenv("NVIDIA_API_KEY"), 
    truncate="NONE", )

In [29]:
from langchain_community.vectorstores import FAISS
vectore_store = FAISS.from_documents(documents, embeddings_model)

In [38]:
similar_result = vectore_store.similarity_search("What is the purpose of the langchain")
similar_result[0].page_content

'Open on GitHub\n\nSetup\n\nLangChain documentation consists of two components:\n\nMain Documentation: Hosted at python.langchain.com, this comprehensive resource serves as the primary user-facing documentation. It covers a wide array of topics, including tutorials, use cases, integrations, and more, offering extensive guidance on building with LangChain. The content for this documentation lives in the /docs directory of the monorepo.\n\nIn-code Documentation: This is documentation of the codebase itself, which is also used to generate the externally facing API Reference. The content for the API reference is autogenerated by scanning the docstrings in the codebase. For this reason we ask that developers document their code well.\n\nThe API Reference is largely autogenerated by sphinx from the code and is hosted by Read the Docs.\n\nWe appreciate all contributions to the documentation, whether it be fixing a typo, adding a new tutorial or example and whether it be in the main documentat

In [39]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template(
    """
    Answer the user follwing qestin based on only on the provided contet:
    
    {context}
    
    
    """
)

In [40]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm_client = ChatNVIDIA(
    model="deepseek-ai/deepseek-r1-distill-llama-8b",
    api_key=os.getenv('NVIDIA_API_KEY'), 
    temperature=0.6,
    top_p=0.7,
    max_tokens=4096,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/langchain_nvidia_ai_endpoints/_common.py:212: UserWarning: Found deepseek-ai/deepseek-r1-distill-llama-8b in available_models, but type is unknown and inference may fail.
  warnings.warn(


In [41]:
document_chain= create_stuff_documents_chain(
llm_client, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the user follwing qestin based on only on the provided contet:\n\n    {context}\n\n\n    '), additional_kwargs={})])
| ChatNVIDIA(base_url='https://integrate.api.nvidia.com/v1', model='deepseek-ai/deepseek-r1-distill-llama-8b', temperature=0.6, max_tokens=4096, top_p=0.7)
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [42]:
from langchain.chains.retrieval import create_retrieval_chain
retriver = vectore_store.as_retriever()
retrival_chain = create_retrieval_chain(retriver, document_chain)
retrival_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'NVIDIAEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79e678d29820>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Answer the user follwing qestin based on only on the provided contet:\n\n    {context}\n\n\n    '), additional_kwargs={})])
            | ChatNVIDI

In [43]:
retrival_chain.invoke({"input":"give me a list of All chat models available"})

{'input': 'give me a list of All chat models available',
 'context': [Document(id='c96521cf-da43-4313-bbc3-1b9355323178', metadata={'source': 'https://python.langchain.com/v0.2/api_reference/_modules/langchain_community/llms/databricks.html'}, page_content='Source code for langchain_community.llms.databricks\n\nimport os\nimport re\nimport warnings\nfrom abc import ABC, abstractmethod\nfrom typing import Any, Callable, Dict, List, Mapping, Optional\n\nimport requests\nfrom langchain_core.callbacks import CallbackManagerForLLMRun\nfrom langchain_core.language_models import LLM\nfrom langchain_core.pydantic_v1 import (\n    BaseModel,\n    Field,\n    PrivateAttr,\n    root_validator,\n    validator,\n)\n\n__all__ = ["Databricks"]\n\n\nclass _DatabricksClientBase(BaseModel, ABC):\n    """A base JSON API client that talks to Databricks."""\n\n    api_url: str\n    api_token: str\n\n    def request(self, method: str, url: str, request: Any) -> Any:\n        headers = {"Authorization": f"Be

In [44]:
try:
    for chunk in retrival_chain.stream({
        "input": "What is the langchain project all about?"
    }):
        if 'answer' in chunk:
            print(chunk['answer'], end="", flush=True)
except Exception as e:
    print(f"Error occurred: {str(e)}")

<think>
Okay, so I need to figure out how to contribute to the LangChain documentation based on the provided content. Let me start by reading through the setup and the documentation sections to understand what's expected.

From the setup, I see that LangChain has both main documentation and in-code documentation. The main documentation is on python.langchain.com and is in the /docs directory. It's written using ipynb and mdx files and built with Docusaurus 2. The in-code documentation is auto-generated from the code using sphinx and is hosted on Read the Docs.

The user wants me to answer a question based on the provided content. Since the user didn't specify a question, I'll assume they might be asking about the process of contributing to the documentation or how to write effective docstrings.

Looking at the in-code documentation section, it's crucial for the API reference. The user emphasizes that all functions, classes, and methods should have well-documented docstrings following t